# Plot XRF and XRD maps from flyscans

This notebook will plot XRF and XRD maps from flyscans. It requires data from the `.h5` masterfile, the `_falconx.h5` file and the `_pilatus_integrated.h5` file.

The user should provide a list of scan IDs, which are read and stitched together.

Two ROIs should be defined in the various cells, one for the interval to use for the XRF maps (in keV) and one for the XRD maps (in scattering units).

NB: The notebook is intended for large dataset and as such, reduce the data size by converting to unsigned integers (*uint32*). For small dataset with low count rate it might be beneficial to switch to *float* for a larger dynamic range.

### Content
[**Import and reshape data**](#import)  
[**Plot average fluorescence spectrum**](#avg_spec)  
[**Plot XRF maps**](#xrf_map)  
[**Plot average diffraction pattern**](#avg_xrd)  
[**Plot XRD maps**](#xrd_map)  

In [1]:
%matplotlib widget
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import DanMAX as DM
from IPython.utils import io

In [3]:
# Define list of scan IDs:

scans = np.arange(xxxx,xxxx+1,dtype=int)

#Select emmission lines to plot:
lines = {'Ca'    : ( 3.4 ,  4.2),
         'Zn'    : ( 8.45,  9.9),
         'Zn_Kb' : ( 9.4 ,  9.7),
         'Sr'    : (13.85, 14.5)}

# 'Ar'    : ( 2.8 ,  3.0 )
# 'Ca'    : ( 3.4 ,  4.2 )
# 'Ca_Ka' : ( 3.5 ,  3.8 )
# 'Ca_Kb' : ( 3.84,  4.04)
# 'Zn'    : ( 8.45,  9.9 )
# 'Zn_Ka' : ( 8.45,  8.70)
# 'Zn_Kb' : ( 9.4 ,  9.7 )
# 'Br'    : (11.6 , 12.1 )
# 'Sr'    : (13.85, 14.5 )

NameError: name 'xxxx' is not defined

#### import and reshape data  <a id='import'></a>

In [2]:
# import falcon x data
#fname = DM.findScan(int(scans[0]))
#with h5py.File(fname,'r') as f:
#    Emax = f['/entry/instrument/pilatus/energy'][()]*10**-3 # keV
# Energy calibration (Conversion of chanels to energy)      
#energy = np.arange(4096)*0.01280573-0.14478


xx, yy, SS, energy, II, x_xrd, Q = DM.stitchScans(scans,XRF=True,XRD=True)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# calculate average spectrum
S_avg = np.nanmean(SS,axis=(0,1))
I_avg = np.mean(II,axis=(0,1))
print(f'Stitched image dimensions: {SS.shape}')

/data/visitors/danmax/20210507/2021120808/raw/club1/scan-0636.h5
Stitched image dimensions: (19, 220, 3018)


In [3]:
print(np.shape(xx),np.shape(yy),np.shape(II))
plt.figure()
plt.plot(yy)


(209,) (319,) (19, 220, 2906)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### plot the average spectrum  <a id='avg_spec'></a>

In [ ]:
plt.figure()
plt.title('Average spectrum')
plt.plot(energy[energy<Emax*1.1],S_avg)
plt.xlabel('Energy (keV)')
plt.ylabel('Intensity')
plt.xlim(0,Emax*1.1)
# plot emmission lines
print('Selected emmission lines:')
for key in lines:
    plt.axvline(lines[key][0],c='k',ls='--',lw=1)
    plt.axvline(lines[key][1],c='k',ls='--',lw=1)
    print(f'{key:<8s}: {lines[key][0]:4.1f} - {lines[key][1]:4.1f} keV')

#### plot XRF heatmaps  <a id='xrf_map'></a>

In [ ]:
# Set the number of columns for the figure
cols = 4 

rows = int(len(lines)/cols) + (len(lines)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

for i,key in enumerate(lines):
    lower, upper = lines[key]
    lower, upper = np.digitize(lower, energy), np.digitize(upper, energy)

    # reshape the fluorescence signal from the falcon x
    #S  = np.mean(falconx[:,lower:upper],axis=1).reshape(map_shape)
    S = np.mean(SS[:,:,lower:upper],axis=2)
    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(xx,
                        yy,
                        S,
                        norm='log',
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel(M2[0]+' mm')
    ax.set_ylabel(M1[0]+' mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(lines)+1):
    fig.delaxes(axs[-i])

#### plot the average diffraction pattern <a id='avg_xrd'></a>

In [8]:
# define ROI dictionary with a label str and a tuple of lower and upper limits, given in the scattering units (2theta or Q)
#     'label': (lower,upper)
rois = {'peak_1' : (  4, 4.1),
        'peak_2' : (14.24, 16.36)}

# plot average XRD pattern
plt.figure()
plt.title('Average XRD pattern')
plt.plot(x_xrd,I_avg)
print(II.dtype)
if Q:
    plt.xlabel('Q (A-1)')
else:
    plt.xlabel('2theta (deg)')
plt.ylabel('Intensity')
plt.yscale('log')
for key in rois:
    plt.axvline(rois[key][0],c='k',ls='--',lw=1)
    plt.axvline(rois[key][1],c='k',ls='--',lw=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

uint32


NameError: name 'Q' is not defined

#### plot XRD maps <a id='xrd_map'></a>

In [9]:
# Set the number of columns for the figure
cols = 4 

rows = int(len(rois)/cols) + (len(rois)%cols!=0)
# initialize subplots with shared x- and y-axes
fig,axs = plt.subplots(rows,cols,sharex=True,sharey=True)
fig.tight_layout()
axs = axs.flatten() # flatten the axes list to make it easier to index

for i,key in enumerate(rois):
    lower, upper = rois[key]
    lower, upper = np.digitize(lower, x_xrd), np.digitize(upper, x_xrd)

    # calculate the average diffraction peak signal from the integrated XRD data
    I_xrd = np.mean(II[:,:,lower:upper],axis=2)
    # subtract linear background
    I_xrd -= np.mean([np.mean(II[:,:,lower-5:lower],axis=2), np.mean(II[:,:,upper+1:upper+6],axis=2)])#.reshape(map_shape)

    # plot the map as a pseudo colormesh
    ax = axs[i]
    ax.set_title(key)
    pcm = ax.pcolormesh(xx,
                        yy,
                        I_xrd,
                        norm='log',
                        shading='nearest')
    fig.colorbar(pcm,ax=ax,aspect=40,pad=0.05)
    ax.set_xlabel(M2[0]+' mm')
    ax.set_ylabel(M1[0]+' mm')
    # set the aspect ratio to equal to give square pixels
    ax.set_aspect('equal')

# delete surplus plots
for i in range(1,cols*rows-len(rois)+1):
    fig.delaxes(axs[-i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Dimensions of C (19, 220) are incompatible with X (209) and/or Y (319); see help(pcolormesh)

NameError: name 'pcolormesh' is not defined

In [12]:
plt.figure()
plt.imshow(np.nanmean(II,2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …